In [11]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from typing import Tuple

import sys
sys.path.append("..")
from utils import print_answer

1. Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [8]:
train = pd.read_csv('datasets/data-logistic.csv',header = None)
y = train[0]
X = train.loc[:, 1:]

2. Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

In [14]:
def calc_w1(X: pd.DataFrame, y: pd.Series, w1: float, w2: float, k: float, C: float) -> float:
    l = len(y)
    S = 0
    for i in range(0, l):
        S += y[i] * X[1][i] * (1.0 - 1.0 / (1.0 + np.exp(-y[i] * (w1*X[1][i] + w2*X[2][i]))))

    return w1 + (k * (1.0 / l) * S) - k * C * w1

def calc_w2(X: pd.DataFrame, y: pd.Series, w1: float, w2: float, k: float, C: float) -> float:
    l = len(y)
    S = 0
    for i in range(0, l):
        S += y[i] * X[2][i] * (1.0 - 1.0 / (1.0 + np.exp(-y[i] * (w1*X[1][i] + w2*X[2][i]))))

    return w2 + (k * (1.0 / l) * S) - k * C * w2

3. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

In [15]:

def gradient_descent(X: pd.DataFrame, y: pd.Series, w1: float = 0.0, w2: float = 0.0,
                     k: float = 0.1, C: float = 0.0, precision: float = 1e-5, max_iter: int = 10000) -> Tuple[float, float]:
    for i in range(max_iter):
        w1_prev, w2_prev = w1, w2
        w1, w2 = calc_w1(X, y, w1, w2, k, C), calc_w2(X, y, w1, w2, k, C)
        if np.sqrt((w1_prev - w1) ** 2 + (w2_prev - w2) ** 2) <= precision:
            break

    return w1, w2


4. Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

In [16]:
w1, w2 = gradient_descent(X, y)
w1_reg, w2_reg = gradient_descent(X, y, C=10.0)

5. Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)). 

In [17]:
def a(X: pd.DataFrame, w1: float, w2: float) -> pd.Series:
    return 1.0 / (1.0 + np.exp(-w1 * X[1] - w2*X[2]))

y_1 = a(X, w1, w2)
y_1_reg = a(X, w1_reg, w2_reg)

auc = roc_auc_score(y, y_1)
auc_reg = roc_auc_score(y, y_1_reg)

print_answer(1, f"{auc:.3f} {auc_reg:.3f}")

0.927 0.936
